In [1]:
from Conv_Layer import Conv_RBS_density_I2_3D
from PennyLane_toolbox import *
from toolbox import *

In [2]:
I = 4
J = 2
K = 4
n = I+I+J
dev = qml.device("default.qubit", wires=n)

@qml.qnode(dev)
def circuit(angle):
    qml.PauliX(wires=0)
    qml.PauliX(wires=I)
    qml.PauliX(wires=2*I)
    RBS(0,1, angle)
    RBS(I,I+1, angle)
    RBS(2*I,2*I+1, angle)
    return qml.density_matrix([i for i in range(n)])

map_3D = map_RBS_I2_3D_bottom(I, J)
initial_rho_3D = map_Computational_Basis_to_Image_3D_Subspace_density(I, J, map_3D, torch.tensor(circuit(torch.pi/2))).unsqueeze(0)
print(initial_rho_3D.shape)

torch.Size([1, 32, 32])


In [3]:
device = torch.device("cpu")
conv1 = Conv_RBS_density_I2_3D(I, K, J, device)
conv_res = conv1(initial_rho_3D)
angles = [float(conv1.Parameters[i]) for i in range(len(conv1.Parameters))]

# We define again the circuit:
Param_dictionary, list_gates = Conv_3D_gates(I,J,K)
@qml.qnode(dev)
def Circuit_global(angles):
    qml.PauliX(wires=0)
    qml.PauliX(wires=I)
    qml.PauliX(wires=2*I)
    RBS(0,1, torch.pi/2)
    RBS(I,I+1, torch.pi/2)
    RBS(2*I,2*I+1, torch.pi/2)
    Conv_RBS_3D(angles, Param_dictionary, list_gates)
    return qml.density_matrix([i for i in range(n)])

penny_res = map_Computational_Basis_to_Image_3D_Subspace_density(I, J, map_3D, torch.tensor(Circuit_global(angles)))
if abs(torch.sum(penny_res - conv_res))<1e-6:
    print("correct!")

correct!
